In [1]:
import datetime as dt
import os
import re
import timeit
from itertools import combinations


import xarray

import basd
import dask
import multiprocessing as mp
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
data_path = '../../../OneDrive - PNNL/Documents/BASD/data/'
pr_sim_hist = xr.open_dataset(data_path + '/pr_day_CESM2-WACCM_historical_r1i1p1f1_gn_20100101-20150101.nc')
pr_obs_hist = xr.open_dataset(data_path + 'pr_day_CESM2-WACCM_historical_r2i1p1f1_gn_20100101-20150101.nc')
pr_sim_fut = xr.open_dataset(data_path + 'pr_day_CESM2-WACCM_ssp370_r1i1p1f1_gn_20950101-21010101.nc')

C:\Users\prim232\virtualenvs\3.9_basd\lib\site-packages\xarray\conventions.py:521: SerializationWarning: variable 'pr' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [3]:
# Making the data smaller for testing purposes
#pr_sim_fut = pr_sim_fut[dict(lat=range(75,115), lon=range(70,110))]
#pr_sim_hist = pr_sim_hist[dict(lat=range(75,115), lon=range(70,110))]
#pr_obs_hist = pr_obs_hist[dict(lat=range(75,115), lon=range(70,110))]
pr_sim_fut = pr_sim_fut[dict(lat=range(20), lon=range(20))]
pr_sim_hist = pr_sim_hist[dict(lat=range(20), lon=range(20))]
pr_obs_hist = pr_obs_hist[dict(lat=range(20), lon=range(20))]

In [4]:
params = basd.Parameters(lower_bound=0,
                         lower_threshold=0.0000011574,
                         trend_preservation='mixed',
                         distribution='gamma')
ba = basd.Adjustment(pr_obs_hist, pr_sim_hist, pr_sim_fut, 'pr', params)

C:\Users\prim232\virtualenvs\3.9_basd\lib\site-packages\xarray\coding\cftime_offsets.py:1130: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  return pd.date_range(
C:\Users\prim232\virtualenvs\3.9_basd\lib\site-packages\xarray\coding\cftime_offsets.py:1130: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  return pd.date_range(
C:\Users\prim232\virtualenvs\3.9_basd\lib\site-packages\xarray\coding\cftime_offsets.py:1130: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  return pd.date_range(


In [51]:
# Trying to find ideal chunk size
times = []
for i in range(1, 5):
    start = timeit.default_timer()
    chunk_result = ba.adjust_bias(lat_chunk_size=i, lon_chunk_size=i, n_jobs=-1)
    stop = timeit.default_timer()
    times.append(start-stop)
    print(f'Time for chunk size {i}: {stop-start}')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   48.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   53.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  1

Time for chunk size 3: 261.4084363000002


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  12 out of  25 | elapsed:  2.6min remaining:  2.8min
[Parallel(n_jobs=-1)]: Done  14 out of  25 | elapsed:  2.8min remaining:  2.2min
[Parallel(n_jobs=-1)]: Done  16 out of  25 | elapsed:  3.0min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  18 out of  25 | elapsed:  3.7min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  20 out of  25 | elapsed:  3

Time for chunk size 4: 261.30030630000056


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  4.4min finished


In [52]:
times

[-242.51730260000022,
 -265.0072931000004,
 -261.4084363000002,
 -261.30030630000056]

In [10]:
chunk_result.to_netcdf(data_path + 'pr_sim_fut_ba.nc')

In [16]:
# Cells we're applying over
i_locations = np.ndindex(5, 5)

# Setting up list of dictionary items to pass to function
locs = []
for i_loc in i_locations:
    loc = dict(lat=i_loc[0], lon=i_loc[1])
    locs.append(loc)

# Vectorize bias adjust one location function
my_func = np.vectorize(ba.adjust_bias_one_location)

In [18]:
start = timeit.default_timer()
vector_out = my_func(locs)
stop = timeit.default_timer()
print(f'Vector Time: {stop-start}')

start = timeit.default_timer()
loop_out = []
for loc in locs:
    loop_out.append(ba.adjust_bias_one_location(loc))
stop = timeit.default_timer()
print(f'Loop Time: {stop-start}')

start = timeit.default_timer()
list_comp_out = [ba.adjust_bias_one_location(loc) for loc in locs]
stop = timeit.default_timer()
print(f'List Comp Time: {stop-start}')

start = timeit.default_timer()
map_out = list(map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Map Time: {stop-start}')

pool = mp.Pool()
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool Map Time: {stop-start}')

C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 49
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 47
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 43
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 42
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 40
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 38
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 34
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\bas

Vector Time: 34.85927319999996


C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 49
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 47
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 43
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 42
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 40
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 38
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 34
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\bas

Loop Time: 35.45779920000018


C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 49
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 47
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 43
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 42
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 40
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 38
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 34
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\bas

List Comp Time: 36.16490810000005


C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 49
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 47
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 43
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 42
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 40
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 38
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\basd\utils.py:1260: UserWarning: due to little input data: reducing n_quantiles to 34
  warnings.warn(msg)
C:\Users\prim232\GitRepos\basd\bas

Map Time: 35.76844750000009
Pool Map Time: 22.768851799999993


In [53]:
pool = mp.Pool(1)
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool 1 Time: {stop-start}')

pool = mp.Pool(4)
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool 4 Time: {stop-start}')

pool = mp.Pool(8)
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool 8 Time: {stop-start}')

pool = mp.Pool(16)
start = timeit.default_timer()
pool_out = list(pool.map(ba.adjust_bias_one_location, locs))
stop = timeit.default_timer()
print(f'Pool 16 Time: {stop-start}')

Pool 1 Time: 39.50112799999988
Pool 4 Time: 16.773764999999912
Pool 8 Time: 18.433224600000358
Pool 16 Time: 24.62542999999914
